# Agentic Sales Pipeline

## Initial Imports

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

## Load API tokens for our 3rd party APIs

In [ ]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [ ]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

## Create Pydantic Models for Structured Output

In [ ]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

## Importing Tools

In [ ]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

## Lead Qualification Crew, Agents and Tasks

In [ ]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()]
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

## Email Engagement Crew

In [ ]:
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist']
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist']
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)

# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True
)

2025-07-26 16:03:03,225 - 140097770802048 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


## Creating Complete Sales Flow

In [ ]:
from crewai import Flow
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads = [
            {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
        ]
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = lead_scoring_crew.kickoff_for_each(leads)
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        return [score for score in scores if score['lead_score'].score > 70]

    @listen(filter_leads)
    def write_email(self, leads):
        scored_leads = [lead.to_dict() for lead in leads]
        emails = email_writing_crew.kickoff_for_each(scored_leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

## Plotting the Flow

In [ ]:
flow.plot()

Graph saved as crewai_flow_graph.html


In [ ]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)

## Flow Kickoff

In [ ]:
emails = await flow.kickoff()

2025-07-26 16:03:06,711 - 140097770802048 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Lead Data Specialist
## Task: Collect and analyze the following information about the lead:
- Personal Information:
  - Name: Obtain the full name of the lead.
  - Job Title: Determine the lead's current job title.
  - Role Relevance: Assess how relevant the lead's role is to the decision-making process on a scale from 0 to 10.
  - Professional Background: Optionally, gather a brief description of the lead's professional background.

- Company Information:
  - Company Name: Identify the name of the company the lead works for.
  - Industry: Determine the industry in which the company operates.
  - Company Size: Estimate the size of the company in terms of employee count.
  - Revenue: If available, collect information on the annual revenue of the company.
  - Market Presence: Evaluate the company's market presence on a scale from 0 to 10.

- Our Company and Product:
  - Company Name: CrewAI
  - Product: Multi-Agent Orchestration Platform
  - ICP: Enterprise companies looking int



# Agent: Cultural Fit Analyst
## Thought: I need to gather information on Clearbit's cultural values and internal culture to assess the cultural alignment between Clearbit and CrewAI, as well as evaluate their strategic alignment with our organization's objectives.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Clearbit company culture values\"}"
## Tool Output: 

Search results: Title: Defining Clearbit's company values
Link: https://clearbit.com/blog/company-values
Snippet: Surfacing values, from the bottom up
---
Title: My first year at Clearbit: learning how to be remote and resourceful
Link: https://clearbit.com/blog/remote-work-growth
Snippet: Autonomy: At Clearbit, I have complete freedom to work across products and teams, to apply my effort where it will be most valuable. · Trust: ...
---
Title: Working at Clearbit | Glassdoor
Link: https://www.glassdoor.com/Overview/Working-at-Clearbit-EI_IE2208101.11,19.htm
Snippet: Fast Paced, Smart coworkers, wor



# Agent: Cultural Fit Analyst
## Thought: Thought: I have gathered Clearbit's cultural values and internal culture details. Now, I will analyze the strategic alignment of Clearbit's goals and mission with CrewAI's organizational objectives.
To do this, I will look for Clearbit's mission statement and strategic goals.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Clearbit company mission statement and strategic goals\"}"
## Tool Output: 

Search results: Title: Defining Clearbit's company values
Link: https://clearbit.com/blog/company-values
Snippet: Clearbit Global Coverage – delivering on our promise to be the best in B2B data. International coverage on companies, contact, and buying intent ...
---
Title: Customer Retention - Clearbit
Link: https://clearbit.com/resources/books/data-driven-marketing/customer-retention
Snippet: The tricky—and perhaps wonderful—characteristic of retention is that its mission spans across roles and disciplines, including marke

 Error parsing LLM output, agent will retry: I did it wrong. Invalid Format: I missed the 'Action:' after 'Thought:'. I will do right next, and don't use a tool I have already used.

If you don't need to use any more tools, you must give your best complete final answer, make sure it satisfy the expect criteria, use the EXACT format below:

Thought: I now can give a great answer
Final Answer: my best complete final answer to the task.




# Agent: Cultural Fit Analyst
## Final Answer: 
**Cultural Alignment Assessment Report for Clearbit**

**Cultural Values and Internal Culture:**
Clearbit has articulated its cultural values based on input from its team, grouping them into the following core principles:

1. **Care (Give a shit)** - Empathy for customers; understanding their frustrations and needs.
2. **Craft (Master it)** - Commitment to continuous learning and improvement in one’s craft.
3. **Team (Work together)** - Emphasis on teamwork and collaboration, breaking down silos.
4. **Tru

...



# Agent: Lead Scorer and Validator
## Thought: Thought: I will read content from the Forbes article, as it is likely to provide a comprehensive overview of Clearbit, including its company profile, which might include information about its market presence and general performance.
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://www.forbes.com/companies/clearbit/\"}"
## Tool Output: 
Clearbit | Company Overview & NewsNewslettersGamesShare a News TipFeaturedFeaturedBreaking NewsWhite House WatchDaily Cover StoriesAI’s Nuanced Impact And A Quest To Quantify ItDNA of SuccessEmbracing And Bracing For AIFacing A Volatile Market, C-Suites Look To The CFO For Strategic GuidanceForbes Research: 2023 Small Business SurveyYour Four-Part Blueprint To Unlock AI Value In 2025 | Paid ProgramBy The Numbers: Meet The Forbes 30 Under 30 Europe Class Of 2025Dell Technologies BrandVoice | Paid ProgramScaling AILocal Leaders, Global Impact | Paid ProgramForbes Top Creators 



# Agent: Lead Scorer and Validator
## Thought: Thought: I was not able to gather useful data from the Forbes page due to extensive distractions from its interface. Therefore, I will search for a simpler overview of Clearbit's company profile to gather the necessary information about its size, market presence, and performance.
## Using tool: Search the internet
## Tool Input: 
"{\"search_query\": \"Clearbit company information 2023 overview business profile\"}"
## Tool Output: 

Search results: Title: Clearbit is Now Breeze Intelligence for HubSpot
Link: https://clearbit.com/
Snippet: Now reinvented with Artificial Intelligence—Clearbit is the first HubSpot Native Data Provider. Enrich your records, score, route, and reveal buying intent ...
---
Title: Clearbit - Overview, News & Similar companies | ZoomInfo.com
Link: https://www.zoominfo.com/c/clearbit/464190707
Snippet: Clearbit, Inc. develops business intelligence to help companies find more information of customers in order to inc

 

Action 'Calculate and compile lead score' don't exist, these are the only available Actions:
Tool Name: Search the internet(**kwargs: Any) -> Any
Tool Description: Search the internet(search_query: 'string') - A tool that can be used to search the internet with a search_query. search_query: 'Mandatory search query you want to use to search the internet'
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'title': 'Search Query', 'type': 'string'}}
Tool Name: Read website content(**kwargs: Any) -> Any
Tool Description: Read website content(website_url: 'string') - A tool that can be used to read a website content. website_url: 'Mandatory website url to read the file'
Tool Arguments: {'website_url': {'description': 'Mandatory website url to read the file', 'title': 'Website Url', 'type': 'string'}}



# Agent: Lead Scorer and Validator
## Thought: Thought: I have consistently encountered blocks in accessing specific website

2025-07-26 16:04:28,387 - 140097770802048 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Email Content Writer
## Task: Craft a highly personalized email using the lead's name, job title, company information, and any relevant personal or company achievements. The email should speak directly to the lead's interests and the needs of their company. This is not as cold outreach as it is a follow up to a lead form, so keep it short and to the point. Don't use any salutations or closing remarks, nor too complex sentences.
Our Company and Product: - Company Name: CrewAI - Product: Multi-Agent Orchestration Platform - ICP: Enterprise companies looking into Agentic automation. - Pitch: We are a platform that allows you to orchestrate AI Agents for automations to any vertical.
Use the following information: Personal Info: {'name': 'João Moura', 'job_title': 'Director of Engineering', 'role_relevance': 10, 'professional_background': None} Company Info: {'company_name': 'Clearbit', 'industry': 'B2B Data and Marketing Solutions', 'company_size': 907, 'revenue': 31500000.0, 'mar

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [ ]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0440


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,293583,284435,9148,66


In [ ]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0003


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,1748,1411,337,3


## Inspecting Results

In [ ]:
scores = flow.state["score_crews_results"]

In [ ]:
import pandas as pd
from IPython.display import display, HTML

lead_scoring_result = scores[0].pydantic

# Create a dictionary with the nested structure flattened
data = {
    'Name': lead_scoring_result.personal_info.name,
    'Job Title': lead_scoring_result.personal_info.job_title,
    'Role Relevance': lead_scoring_result.personal_info.role_relevance,
    'Professional Background': lead_scoring_result.personal_info.professional_background,
    'Company Name': lead_scoring_result.company_info.company_name,
    'Industry': lead_scoring_result.company_info.industry,
    'Company Size': lead_scoring_result.company_info.company_size,
    'Revenue': lead_scoring_result.company_info.revenue,
    'Market Presence': lead_scoring_result.company_info.market_presence,
    'Lead Score': lead_scoring_result.lead_score.score,
    'Scoring Criteria': ', '.join(lead_scoring_result.lead_score.scoring_criteria),
    'Validation Notes': lead_scoring_result.lead_score.validation_notes
}

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Value'])

# Reset the index to turn the original column names into a regular column
df = df.reset_index()

# Rename the index column to 'Attribute'
df = df.rename(columns={'index': 'Attribute'})

# Create HTML table with bold attributes and left-aligned values
html_table = df.style.set_properties(**{'text-align': 'left'}) \
                     .format({'Attribute': lambda x: f'<b>{x}</b>'}) \
                     .hide(axis='index') \
                     .to_html()

# Display the styled HTML table
display(HTML(html_table))

Attribute,Value
Name,João Moura
Job Title,Director of Engineering
Role Relevance,10
Professional Background,None
Company Name,Clearbit
Industry,B2B Data and Marketing Solutions
Company Size,907
Revenue,31500000.000000
Market Presence,10
Lead Score,100


## Results

In [ ]:
import textwrap

result_text = emails[0].raw
wrapped_text = textwrap.fill(result_text, width=80)
print(wrapped_text)

CrewAI's Multi-Agent Orchestration Platform is designed to boost your automation
capabilities at Clearbit. Automate processes seamlessly across multiple
verticals and enhance your data management efficiency. Ready to see it in
action? Schedule a 15-minute demo with us now [link to calendar]. Want more
insights? Check out this case study [link to case study] showcasing how we
transformed automation for similar enterprises. Your innovation journey with AI
agents starts here—don’t miss out!


## How Complex Can it Get?

In [ ]:
from crewai import Flow
from crewai.flow.flow import listen, start, and_, or_, router

class SalesPipeline(Flow):

  @start()
  def fetch_leads(self):
    # Pull our leads from the database
    # This is a mock, in a real-world scenario, this is where you would
    # fetch leads from a database
    leads = [
      {
        "lead_data": {
          "name": "João Moura",
          "job_title": "Director of Engineering",
          "company": "Clearbit",
          "email": "joao@clearbit.com",
          "use_case": "Using AI Agent to do better data enrichment."
        },
      },
    ]
    return leads

  @listen(fetch_leads)
  def score_leads(self, leads):
    scores = lead_scoring_crew.kickoff_for_each(leads)
    self.state["score_crews_results"] = scores
    return scores

  @listen(score_leads)
  def store_leads_score(self, scores):
    # Here we would store the scores in the database
    return scores

  @listen(score_leads)
  def filter_leads(self, scores):
    return [score for score in scores if score['lead_score'].score > 70]

  @listen(and_(filter_leads, store_leads_score))
  def log_leads(self, leads):
    print(f"Leads: {leads}")

  @router(filter_leads, paths=["high", "medium", "low"])
  def count_leads(self, scores):
    if len(scores) > 10:
      return 'high'
    elif len(scores) > 5:
      return 'medium'
    else:
      return 'low'

  @listen('high')
  def store_in_salesforce(self, leads):
    return leads

  @listen('medium')
  def send_to_sales_team(self, leads):
    return leads

  @listen('low')
  def write_email(self, leads):
    scored_leads = [lead.to_dict() for lead in leads]
    emails = email_writing_crew.kickoff_for_each(scored_leads)
    return emails

  @listen(write_email)
  def send_email(self, emails):
    # Here we would send the emails to the leads
    return emails

## Plotting the Flow

In [ ]:
flow = SalesPipeline()
flow.plot()

Graph saved as crewai_flow_graph.html


In [ ]:
from IPython.display import IFrame

IFrame(src='./crewai_flow_complex.html', width='150%', height=600)